In [2]:
import networkx as nx
import random
import numpy as np


In [3]:
G = nx.watts_strogatz_graph(n = 1000, k = 10, p = 0.5, seed = 10)

In [10]:
class selection_strategies:    
    def __init__(self):
        pass
    def degree_ranking(G, d)->list:
        degree = sorted([(v,(G.degree(v))) for v in G], key=lambda tup: tup[1], reverse=True)
        return degree[:d]
    def closeness_ranking(G, d)->list:
        closeness = nx.closeness_centrality(G)
        degree = sorted([(k,v) for k,v in zip(closeness.keys(), closeness.values())] , key= lambda tup: tup[1], reverse=True)
        return degree[:d]
    def random_ranking(G, d)->list:
        sample = random.sample(list(G.nodes),d)
        sample = [(node, G.degree(node)) for node in sample]
        return sample
    

class nx_landmarks:
    def __init__(self, G, d = 1, selection_strategie = "rand"):
        if type(G) != nx.classes.graph.Graph:
            raise AttributeError(
                f"Expected graph to be of type networkx.classes.graph.Graph. Recieved {type(G)}"
            )
        self.graph = G
        if type(d) != int:
            raise AttributeError(
                f"Expected d to be of type int. Recieved {type(d)}"
            )
        self.d = d
        supported_rankings = {
            'rand': selection_strategies.random_ranking,
            'deg': selection_strategies.degree_ranking,
            'close': selection_strategies.closeness_ranking,
        }
        if selection_strategie not in supported_rankings.keys():
            raise AttributeError(
                f"Expected selection_strategie to be of type str. Recieved {type(selection_strategie)}"
            )
        self.selection_strategie = supported_rankings[selection_strategie]
        self.landmarks = self.selection_strategie(G,d)
        self.embeddings = self.get_embeddings()
    
    def get_embeddings(self):
        embeddings = np.zeros((self.graph.number_of_nodes(),len(self.landmarks)))
        for ranking, landmark in enumerate(self.landmarks):
            shortest_paths = nx.single_source_shortest_path_length(G, source = landmark[0])
            for node, length in shortest_paths.items():
                embeddings[node-1,ranking] = length
        return embeddings
    
    def shortest_path_estimation_upper_bound(self, source , target):
        uppers = self.embeddings[source-1] + self.embeddings[target-1]
        return min(uppers)
        
    def shortest_path_estimation_lower_bound(self, source , target):
        lowers = self.embeddings[source-1] - self.embeddings[target-1]
        return max(abs(lowers))
        
                


        
    

In [11]:
D = nx_landmarks(G,100,'rand')

In [12]:
D.shortest_path_estimation_upper_bound(source=1, target=748)

np.float64(4.0)

In [13]:
nx.shortest_path_length(G,source=1, target=748)

4

In [14]:
D.embeddings

array([[4., 3., 4., ..., 4., 3., 3.],
       [4., 4., 4., ..., 4., 3., 3.],
       [4., 4., 4., ..., 3., 3., 3.],
       ...,
       [3., 2., 4., ..., 4., 2., 4.],
       [4., 4., 4., ..., 4., 4., 4.],
       [4., 3., 3., ..., 3., 3., 3.]], shape=(1000, 100))

In [9]:
G.degree(1)

13